# Projeto Automação Web - Busca de Preços

## Realizando as importações das bibliotecas

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.common.keys import Keys

import pandas as pd
import shutil
from pathlib import Path
from datetime import date



servico = Service(r'/home/lucasluis/anaconda3/lib/python3.9/chromedriver')
driver = webdriver.Chrome(service=servico)

## Importando a base de dados

In [2]:
Buscas_df = pd.read_excel(r'/home/lucasluis/Python Impressionador/Projeto de Automação Web/buscas.xlsx')
display(Buscas_df)

data_atual = date.today()

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,Iphone 12 64 gb,mini watch,3600,4500
1,RTX 3060,zotac galax,3100,4300
2,geladeira brastemp,usado electrolux,1500,3000
3,smart TV,usado samsung,1600,3200
4,Ideapad 3 8 gb 256 gb ssd,gaming usado,3000,3700
5,samsung s21 128 gb,ultra fe,2900,3600
6,motorola edge 20,pro lite,2000,3000


## Definição das funções de busca pela web

In [3]:
def buscaGoogleShopping(driver,produto,termos_banidos,preço_minimo,preço_maximo,data_atual):

    # Abre o navegador
    driver.get('https://shopping.google.com/')

    #tratamento dos parametros
    data_atual = date.today()
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()

    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')


    driver.find_element(By.XPATH,'//*[@id="REsRA"]').send_keys(produto)
    driver.find_element(By.XPATH,'//*[@id="REsRA"]').send_keys(Keys.ENTER)
    
    
    # Lista de resultados do google shopping
    lista_resultados_busca = driver.find_elements(By.CLASS_NAME,'i0X6df')

    # Para cada resultado será verificado se o resultado corresponde com todas as condições
    
    lista_ofertas = [] # Lista que a função vai retornar com todos os resultados verificados

    for resultado in lista_resultados_busca:
    
        nome = resultado.find_element(By.CLASS_NAME,'Xjkr3b').text
        nome = nome.lower()


        # verficaca se a os termos banidos nos nomes dos produtos
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        # Verifica se o nome do produto corresponde com o produto pedido
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False


        # verificação final do nome
        if not tem_termos_banidos and tem_todos_termos_produto:
            # Tratamento do preço
            preço = resultado.find_element(By.CLASS_NAME,'a8Pemb').text
            preço = preço.replace('R$','').replace(' ','').replace('.','').replace(',','.')
            if '+impostos' in preço:
                preço = preço.replace('+impostos','')
            preço = float(preço)

            #verificando se o preço da dentro do minimo e maximo
            preço_maximo = float(preço_maximo)
            preço_minimo = float(preço_minimo)
            if preço_minimo <= preço <= preço_maximo:
                elemento_link = resultado.find_element(By.CLASS_NAME,'bONr3b')
                elemento_pai = elemento_link.find_element(By.XPATH,'..')
                link = elemento_pai.get_attribute('href')

                # Atualização da lista de ofertas
                lista_ofertas.append((nome,data_atual,preço,link))
        
    return lista_ofertas



def buscaBuscape(driver,produto,termos_banidos,preço_minimo,preço_maximo, data_atual):

    # Tratamento dos parametros
    preço_maximo = float(preço_maximo)
    preço_minimo = float(preço_minimo)
    produto = produto.lower()
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_termos_produto = produto.split(' ')
    data_atual = date.today()

    # Abre o navegador
    driver.get('https://www.buscape.com.br/')
    
    # Pesquisa o produto
    driver.find_element(By.XPATH,'/html/body/div[1]/header/div[1]/div/div/div[3]/div/div/div[1]/input').send_keys(produto, Keys.ENTER)
  
    # Lista de resultados
    
    sleep(10)
    lista_de_resultados = driver.find_elements(By.CLASS_NAME,'Cell_CellBody__YODBS')
    
    lista_ofertas = []
    
    for resultado in lista_de_resultados:
        
        link = resultado.find_element(By.CLASS_NAME,'Cell_Content__fT5st').get_attribute('href')
        nome = resultado.find_element(By.CLASS_NAME,'Cell_Content__fT5st').get_attribute('title')
        nome = nome.lower()


        # verficaca se a os termos banidos nos nomes dos produtos
        tem_termos_banidos = False
        for palavra in lista_termos_banidos:
            if palavra in nome:
                tem_termos_banidos = True

        # Verifica se o nome do produto corresponde com o produto pedido
        tem_todos_termos_produto = True
        for palavra in lista_termos_produto:
            if palavra not in nome:
                tem_todos_termos_produto = False

        if not tem_termos_banidos and tem_todos_termos_produto:
            # Tratamento do preço
            preço = resultado.find_element(By.CLASS_NAME,'CellPrice_Block__2KAo4').text
            preço = preço.replace('Menor preço','')
            preço = preço.replace('R$','').replace(' ','').replace('.','').replace(',','.')
            if 'Por' in preço:
                preço = preço.replace('Por','')
            preço = float(preço)


            if preço_minimo <= preço <= preço_maximo:
                lista_ofertas.append((nome,data_atual,preço,link))
    
        
    return lista_ofertas

# Elaboração dos dataframes com as ofertas e Exportando-os para uma planilha excel

In [4]:
# Criando uma pasta principal para abrigar todas as outras
try:
    Path('/home/lucasluis/Python Impressionador/Projeto de Automação Web/Produtos').mkdir()
except FileExistsError:
    pass

for linha in Buscas_df.index:
    # criação de pasta para cada produto
    try:
        produto = Buscas_df['Nome'].iloc[linha]
        Path(f'/home/lucasluis/Python Impressionador/Projeto de Automação Web/Produtos/{produto}').mkdir()
    except FileExistsError:
        continue
    
    produto = Buscas_df.loc[linha,'Nome']
    termos_banidos = Buscas_df.loc[linha,'Termos banidos']
    preço_minimo = Buscas_df.loc[linha,'Preço mínimo']
    preço_maximo = Buscas_df.loc[linha,'Preço máximo']

    lista_ofertas_google_shopping= buscaGoogleShopping(driver,produto,termos_banidos,preço_minimo,preço_maximo,data_atual)
    
    # Verificação se há itens na lista
    if lista_ofertas_google_shopping:
        # transformando a lista em um dataframe
        Ofertas_google_shopping_df = pd.DataFrame(lista_ofertas_google_shopping, columns=['Nome','Data', 'Preço','Link de Compra'])
        # Exportando o dataframe para uma planilha excel 
        Ofertas_google_shopping_df.to_excel('/home/lucasluis/Python Impressionador/Projeto de Automação Web/Produtos/{}/{}_Google shopping.xlsx'.format(produto,produto), index=False)

    
    lista_ofertas_buscape = buscaBuscape(driver,produto,termos_banidos,preço_minimo,preço_maximo,data_atual)
    
    # Verificação se há itens na lista
    if lista_ofertas_buscape:
        #Transformando a lista em um dataframe
        Ofertas_buscape_df = pd.DataFrame(lista_ofertas_buscape, columns=['Nome','Data', 'Preço','Link de Compra'] )
        # Exportando o dataframe para uma planilha excel 
        Ofertas_buscape_df.to_excel('/home/lucasluis/Python Impressionador/Projeto de Automação Web/Produtos/{}/{}_Buscapé.xlsx'.format(produto,produto),index=False)

driver.quit()

## Comprimindo a pasta com todos os arquivos de oferta

In [5]:
# Comprimindo a pasta principal para possibilitar o envio por e-mail
shutil.make_archive('Produtos', 'zip', '/home/lucasluis/Python Impressionador/Projeto de Automação Web/', 'Produtos')

'/home/lucasluis/Python Impressionador/Projeto de Automação Web/Produtos.zip'